In [ ]:
import itertools
import pandas as pd
import torch
from exp.exp_long_term_forecast import Exp_Long_Term_Forecast
from argparse import Namespace

# --- Define hyperparameter grid ---
param_grid = {
    "seq_len": [512],
    "e_layers": [2],
    "down_sampling_layers": [3],
    "down_sampling_window": [2],
    "learning_rate": [0.01, 0.001],
    "d_model": [16, 32],
    "d_ff": [32, 64],
    "train_epochs": [10],
    "patience": [10],
    "pred_len": [5, 21, 63, 126],
    "data": ["OPTION", "BTCF"]
}

# --- Utility to expand grid ---
def param_combinations(grid):
    keys = grid.keys()
    vals = grid.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

# --- Results storage ---
results = []

# --- Run experiments ---
for params in param_combinations(param_grid):
    # dataset-specific args
    if params["data"] == "OPTION":
        target, channel_num = "t", 22
    elif params["data"] == "BTCF":
        target, channel_num = "taker_buy_volume_spot", 12
    else:
        continue

    setting = f"{params['data']}_{params['seq_len']}_{params['pred_len']}_exp"

    # Build args object (match what your script expects)
    args = Namespace(
        task_name="long_term_forecast",
        is_training=1,
        root_path="./dataset/FBD/",
        data_path=f"{params['data']}.csv",
        model_id=setting,
        model="TimeMixer",
        data="custom",
        features="M",
        seq_len=params["seq_len"],
        label_len=0,
        pred_len=params["pred_len"],
        e_layers=params["e_layers"],
        enc_in=channel_num,
        c_out=channel_num,
        des="Exp",
        target=target,
        itr=1,
        d_model=params["d_model"],
        d_ff=params["d_ff"],
        learning_rate=params["learning_rate"],
        train_epochs=params["train_epochs"],
        patience=params["patience"],
        batch_size=128,
        down_sampling_layers=params["down_sampling_layers"],
        down_sampling_method="avg",
        down_sampling_window=params["down_sampling_window"],
        use_amp=False,
        use_multi_gpu=False,
        use_gpu=torch.cuda.is_available(),
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        checkpoints="./checkpoints/",
        inverse=False,
        use_dtw=False
    )

    exp = Exp_Long_Term_Forecast(args)
    exp.train(setting)
    mse, mae, msIC, msIR = exp.test(setting, test=1)

    results.append({
        "data": params["data"],
        "pred_len": params["pred_len"],
        "seq_len": params["seq_len"],
        "e_layers": params["e_layers"],
        "d_model": params["d_model"],
        "d_ff": params["d_ff"],
        "lr": params["learning_rate"],
        "mse": mse,
        "mae": mae,
        "msIC": msIC,
        "msIR": msIR
    })

# --- Save results ---
df = pd.DataFrame(results)
df.to_csv("experiment_results.csv", index=False)
df